In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib as mpl
import os
from matplotlib.lines import Line2D
from collections import Counter
import math
from sklearn.decomposition import PCA
from aging.behavior.syllables import relabel_by_usage
from tqdm import tqdm
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import random
import scipy
from scipy import stats
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots

In [2]:
format_plots()
#format_pizza_plots()

In [3]:
from matplotlib.colors import LinearSegmentedColormap
# Define the color map
colors = ['#fee6ce','#d94801']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmf=custom_cmap

colors = ['#c7eae5','#008C8D']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmm=custom_cmap

In [4]:
## update data
def filter_df(df):
    max=39
    syll=df.columns.values[df.columns.values>max]
    df.drop(syll,axis=1, inplace=True)
    return df[~df.index.get_level_values('session_name').str.contains('CRL')]

In [5]:
## female change in rate by male arrangement

In [6]:
path = Path('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/dana_ontogeny_females_relabeled_usage_matrix_v00.parquet')
df_dana = pd.read_parquet(path)

path = Path('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/ontogeny_females_relabeled_usage_matrix_v00.parquet')
df = pd.read_parquet(path)

In [7]:
# arrange data
data = df.astype(float, errors='ignore')
data= filter_df(data).groupby(['age','uuid']).mean()
data['data'] = 'min'
data.set_index('data', inplace=True, append=True)

data_dana = df_dana.astype(float, errors='ignore')
data_dana= filter_df(data_dana).groupby(['age','uuid']).mean()
data_dana['data'] = 'dana'
data_dana.set_index('data', inplace=True, append=True)

data_all = pd.concat([data,data_dana])
# choose 16 mice from all dataset
random_seed = 0
n=16
data_all = data_all.groupby('age').apply(lambda x: x.sample(n=n,replace=False,random_state=random_seed) if len(x)>n else x.sample(n=len(x),replace=False,random_state=random_seed)).reset_index(level=0, drop=True)
data_all = data_all[sorted(data_all.columns)]

In [8]:
data = data_all.copy()
data_labels = data.index.get_level_values('age').to_numpy()

In [9]:
temp = data_all.copy()
temp.reset_index(inplace=True)

if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

In [10]:
# for all syllables
syll = data_all.columns
n=len(syll)
s=[]
p=[]
h=[]
a = 0.05

for i in syll:
    # Melt the DataFrame
    melted_df = pd.melt(temp, id_vars='age', value_vars=[i], value_name='value')
    melted_df.drop(columns=['variable'], inplace=True)
    h_statistic, p_value = stats.kruskal(*[group['value'] for name, group in melted_df.groupby('age')])

    # Print the results
    print("H-statistic:", h_statistic)
    print("p-value:", p_value)
    cp = p_value*n
    if cp<0.05:
        s.extend([i])
        p.extend([cp])
        h.extend([h])

H-statistic: 162.5520467449705
p-value: 1.02285400246249e-21
H-statistic: 125.75326408627802
p-value: 4.8374034645542244e-15
H-statistic: 106.99325030836928
p-value: 8.585340002633518e-12
H-statistic: 126.41751216084572
p-value: 3.6924495136641796e-15
H-statistic: 101.45987125628926
p-value: 7.329431354086293e-11
H-statistic: 128.39272245957275
p-value: 1.6508491202834722e-15
H-statistic: 123.04678384575514
p-value: 1.44905486835788e-14
H-statistic: 61.46056901386293
p-value: 0.00010637185985431817
H-statistic: 105.61889653875414
p-value: 1.4667691902107613e-11
H-statistic: 125.15647297348735
p-value: 6.164233833516544e-15
H-statistic: 102.7685959345034
p-value: 4.4266024893223015e-11
H-statistic: 233.5177182584873
p-value: 2.924469812495588e-35
H-statistic: 248.5804822372166
p-value: 3.296334852720808e-38
H-statistic: 91.71044328809899
p-value: 2.9458889088789076e-09
H-statistic: 104.84810845945245
p-value: 1.9790172993448933e-11
H-statistic: 122.97860386582667
p-value: 1.489556786958

In [11]:
print(len(s))

39


In [12]:
s

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]